In [103]:
## 대사 추출하기
import re
import csv
import io

In [104]:
def extract_dialogue_from_txt(script_text):
    """
    주어진 스크립트 텍스트(.txt)에서 대사만 추출하여 리스트로 반환합니다.
    형식: 이름이 한 줄에 나오고 다음 줄부터 대사가 나오는 형식 및
          이름 : 대사 형식 등을 처리하려고 시도합니다.

    Args:
        script_text (str): 영화 대본 텍스트.

    Returns:
        list: [{'Character': 이름, 'Dialogue': 대사}, ...] 형식의 딕셔너리 리스트.
              추출 실패 시 빈 리스트 반환.
    """
    dialogues = []
    lines = script_text.strip().split('\n')

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # 비어있거나 씬 넘버(#숫자.) 인 경우 건너뜀
        if not line or line.startswith('#') or re.match(r'^#\d+\.', line):
            i += 1
            continue

        # 이름 후보: 주로 한글, 짧은 길이, 콜론(:) 없음, 괄호 설명 가능성 (V.O) 등
        # 이 부분은 대본 형식에 따라 매우 가변적이므로, 완벽하지 않을 수 있습니다.
        # 여기서는 이름이 단독 라인에 있을 가능성을 먼저 봅니다.
        potential_name_match = re.match(r'^([가-힣]+(?:\s*\([^)]+\))?)$', line)

        if potential_name_match and (i + 1 < len(lines)):
            character_name = potential_name_match.group(1).strip()
            # 다음 줄이 대사인지 확인 (빈 줄이 아니고, 다른 이름/씬넘버 패턴이 아닌 경우)
            next_line_index = i + 1
            while next_line_index < len(lines) and not lines[next_line_index].strip():
                next_line_index += 1 # 빈 줄 건너뛰기

            if next_line_index < len(lines):
                next_line = lines[next_line_index].strip()
                is_next_line_name = bool(re.match(r'^([가-힣]+(?:\s*\([^)]+\))?)$', next_line))
                is_scene_heading = next_line.startswith('#') or re.match(r'^#\d+\.', next_line)

                # 다음 줄이 이름이나 씬넘버가 아니라면 대사로 간주
                if not is_next_line_name and not is_scene_heading:
                    current_dialogue_lines = []
                    dialogue_line_index = next_line_index
                    while dialogue_line_index < len(lines):
                        dialogue_part = lines[dialogue_line_index].strip()
                        # 다음 줄이 새로운 이름/씬넘버/빈줄이 아니면 대사 연속으로 간주
                        is_dialogue_name_follow = bool(re.match(r'^([가-힣]+(?:\s*\([^)]+\))?)$', dialogue_part))
                        is_scene_heading_follow = dialogue_part.startswith('#') or re.match(r'^#\d+\.', dialogue_part)

                        if not dialogue_part or is_dialogue_name_follow or is_scene_heading_follow:
                            break # 대사 끝

                        current_dialogue_lines.append(dialogue_part)
                        dialogue_line_index += 1

                    if current_dialogue_lines:
                        full_dialogue = " ".join(current_dialogue_lines).strip()
                        # 괄호 안 지문/설명 제거 (예: (웃으며) 제거)
                        cleaned_dialogue = re.sub(r'\([^)]*\)', '', full_dialogue).strip()
                        # 여러 공백을 하나로 변경
                        cleaned_dialogue = ' '.join(cleaned_dialogue.split())

                        if cleaned_dialogue:
                            dialogues.append({
                                'Character': character_name,
                                'Dialogue': cleaned_dialogue
                            })
                        i = dialogue_line_index # 다음 처리할 라인 인덱스 업데이트
                        continue # 다음 루프로

        # 이름 : 대사 형식도 확인
        dialogue_colon_match = re.match(r'^\s*([^:\n]+?)\s*:\s*(.*)', line)
        if dialogue_colon_match:
             character_name = dialogue_colon_match.group(1).strip()
             dialogue_text = dialogue_colon_match.group(2).strip()
             # 괄호 안 지문/설명 제거
             cleaned_dialogue = re.sub(r'\([^)]*\)', '', dialogue_text).strip()
             cleaned_dialogue = ' '.join(cleaned_dialogue.split())
             if cleaned_dialogue:
                 dialogues.append({
                     'Character': character_name,
                     'Dialogue': cleaned_dialogue
                 })
             i += 1
             continue # 다음 루프로

        # 위 패턴에 맞지 않으면 다음 줄로 이동
        i += 1

    return dialogues


In [105]:
def extract_dialogue_tab_format(script_text):
    """
    주어진 스크립트 텍스트에서 대사만 추출하여 리스트로 반환합니다.
    형식: '이름\t대사' 및 '\t대사' (연속 줄) 형식을 처리합니다.

    Args:
        script_text (str): 영화 대본 텍스트.

    Returns:
        list: [{'Character': 이름, 'Dialogue': 대사}, ...] 형식의 딕셔너리 리스트.
              추출 실패 시 빈 리스트 반환.
    """
    dialogues = []
    lines = script_text.strip().split('\n')

    current_character = None
    current_dialogue_lines = []

    # 이름 TAB 대사 패턴
    dialogue_start_re = re.compile(r'^([^\t\n]+)\t(.*)')
    # TAB 대사 (연속) 패턴
    continuation_re = re.compile(r'^\t(.*)')
    # 씬 넘버 패턴 (숫자. 으로 시작)
    scene_re = re.compile(r'^\d+\.\s+')
    # 괄호 안의 지문/설명 제거 패턴
    action_re = re.compile(r'\([^)]*\)')

    for line in lines:
        line_stripped = line.strip() # 앞뒤 공백 제거 후 내용 확인용
        
        # 빈 줄 또는 씬 넘버 건너뛰기
        if not line_stripped or scene_re.match(line):
            # 이전 대사가 있었다면 저장
            if current_character and current_dialogue_lines:
                full_dialogue = " ".join(current_dialogue_lines).strip()
                cleaned_dialogue = action_re.sub('', full_dialogue).strip()
                cleaned_dialogue = ' '.join(cleaned_dialogue.split())
                if cleaned_dialogue:
                    dialogues.append({
                        'Character': current_character,
                        'Dialogue': cleaned_dialogue
                    })
            current_character = None
            current_dialogue_lines = []
            continue

        match_start = dialogue_start_re.match(line) # 원본 라인으로 매칭 (탭 확인 위해)
        match_cont = continuation_re.match(line) # 원본 라인으로 매칭 (탭 확인 위해)

        if match_start:
            # 이전 대사가 있었다면 저장
            if current_character and current_dialogue_lines:
                full_dialogue = " ".join(current_dialogue_lines).strip()
                cleaned_dialogue = action_re.sub('', full_dialogue).strip()
                cleaned_dialogue = ' '.join(cleaned_dialogue.split())
                if cleaned_dialogue:
                    dialogues.append({
                        'Character': current_character,
                        'Dialogue': cleaned_dialogue
                    })
            
            # 새 대사 시작
            current_character = match_start.group(1).strip()
            initial_dialogue = match_start.group(2).strip()
            current_dialogue_lines = [initial_dialogue] if initial_dialogue else []

        elif match_cont and current_character:
            # 대사 연속
            continuation_text = match_cont.group(1).strip()
            if continuation_text:
                current_dialogue_lines.append(continuation_text)
                
        else: # 지문 또는 다른 형식의 라인
            # 이전 대사가 있었다면 저장
            if current_character and current_dialogue_lines:
                full_dialogue = " ".join(current_dialogue_lines).strip()
                cleaned_dialogue = action_re.sub('', full_dialogue).strip()
                cleaned_dialogue = ' '.join(cleaned_dialogue.split())
                if cleaned_dialogue:
                    dialogues.append({
                        'Character': current_character,
                        'Dialogue': cleaned_dialogue
                    })
            current_character = None
            current_dialogue_lines = []

    # 마지막 대사 블록 처리
    if current_character and current_dialogue_lines:
        full_dialogue = " ".join(current_dialogue_lines).strip()
        cleaned_dialogue = action_re.sub('', full_dialogue).strip()
        cleaned_dialogue = ' '.join(cleaned_dialogue.split())
        if cleaned_dialogue:
            dialogues.append({
                'Character': current_character,
                'Dialogue': cleaned_dialogue
            })

    return dialogues

In [106]:
def extract_dialogue_colon_format(script_text):
    """
    주어진 스크립트 텍스트에서 대사만 추출하여 리스트로 반환합니다.
    형식: '이름 : 대사' 형식을 주로 처리합니다.

    Args:
        script_text (str): 영화 대본 텍스트.

    Returns:
        list: [{'Character': 이름, 'Dialogue': 대사}, ...] 형식의 딕셔너리 리스트.
              추출 실패 시 빈 리스트 반환.
    """
    dialogues = []
    lines = script_text.strip().split('\n')

    # 이름 : 대사 패턴 (이름에 괄호 설명 포함 가능)
    dialogue_re = re.compile(r'^\s*([^:\n]+?)\s*:\s*(.*)')
    # 씬 넘버 패턴
    scene_re = re.compile(r'^S#\s*\d+')
    # 괄호 안의 지문/설명 제거 패턴
    action_re = re.compile(r'\([^)]*\)')

    for line in lines:
        line_stripped = line.strip()

        # 빈 줄 또는 씬 넘버 건너뛰기
        if not line_stripped or scene_re.match(line):
            continue

        match = dialogue_re.match(line)
        if match:
            character_name = match.group(1).strip()
            dialogue_text = match.group(2).strip()

            # 이름에 포함된 (V.O), (off) 등은 유지할 수 있으나,
            # 대사 중의 (놀란듯), (엔지니어에게) 등은 제거
            cleaned_dialogue = action_re.sub('', dialogue_text).strip()
            # 여러 공백을 하나로 정리
            cleaned_dialogue = ' '.join(cleaned_dialogue.split())

            if cleaned_dialogue: # 내용이 있는 경우만 추가
                dialogues.append({
                    'Character': character_name,
                    'Dialogue': cleaned_dialogue
                })
        # else:
            # 연속 줄 처리 로직 추가 가능 (들여쓰기 등 기준 필요)
            # 이 예제에서는 '이름 : 대사' 형식만 처리합니다.

    return dialogues

In [107]:
def extract_dialogue_indented_final_v3(script_text):
    """
    주어진 스크립트 텍스트에서 대사만 추출하여 리스트로 반환합니다. (들여쓰기 활용 및 연속줄 처리 개선)
    형식: '    이름    대사' 형식 및 연속 줄 대사를 처리합니다.
          continuation_indent_re 변수를 실제로 활용합니다.

    Args:
        script_text (str): 영화 대본 텍스트.

    Returns:
        list: [{'Character': 이름, 'Dialogue': 대사}, ...] 형식의 딕셔너리 리스트.
    """
    dialogues = []
    # 원본 줄바꿈 유지, 페이지 넘김 문자는 제거
    lines = script_text.replace('\f', '').split('\n')

    # 이름+대사 시작 패턴: ^(들여쓰기)(이름)(공백+)(대사)
    dialogue_start_re = re.compile(r'^(\s+)([^\s].*?[^\s](?:\s*\([^)]+\))?)\s+(.+)')
    # 이름 없이 들여쓰기만 된 연속 대사/지문 패턴
    continuation_indent_re = re.compile(r'^(\s+)(.*)') # 들여쓰기 된 모든 줄
    # 씬 넘버 패턴
    scene_re = re.compile(r'^\d+\.\s+')
    # 괄호 안 지문 제거 패턴
    action_re = re.compile(r'\([^)]*\)')
    # 빈 줄 확인용
    empty_line_re = re.compile(r'^\s*$')

    current_character = None
    current_dialogue_lines = []
    last_indent_level = -1 # 마지막 대사 시작 줄의 들여쓰기 레벨

    for line in lines:
        stripped_line = line.strip()

        # 빈 줄 처리
        if not stripped_line:
            if current_character and current_dialogue_lines:
                full_dialogue = " ".join(current_dialogue_lines).strip()
                full_dialogue = ' '.join(full_dialogue.split())
                if full_dialogue:
                    dialogues.append({'Character': current_character, 'Dialogue': full_dialogue})
            current_character = None
            current_dialogue_lines = []
            last_indent_level = -1
            continue

        # 씬 넘버 처리
        if scene_re.match(stripped_line):
            if current_character and current_dialogue_lines:
                 full_dialogue = " ".join(current_dialogue_lines).strip()
                 full_dialogue = ' '.join(full_dialogue.split())
                 if full_dialogue:
                     dialogues.append({'Character': current_character, 'Dialogue': full_dialogue})
            current_character = None
            current_dialogue_lines = []
            last_indent_level = -1
            continue

        match_start = dialogue_start_re.match(line) # 이름 + 대사 시작 확인

        if match_start:
            # 이전 대사 저장
            if current_character and current_dialogue_lines:
                full_dialogue = " ".join(current_dialogue_lines).strip()
                full_dialogue = ' '.join(full_dialogue.split())
                if full_dialogue:
                    dialogues.append({'Character': current_character, 'Dialogue': full_dialogue})

            # 새 대사 시작
            current_indent = len(match_start.group(1))
            current_character = match_start.group(2).strip()
            initial_dialogue = match_start.group(3).strip()
            cleaned_part = action_re.sub('', initial_dialogue).strip()
            current_dialogue_lines = [cleaned_part] if cleaned_part else []
            last_indent_level = current_indent

        elif current_character: # 현재 화자가 있는 상태에서
            cont_match = continuation_indent_re.match(line) # <<-- continuation_indent_re 사용
            if cont_match:
                 current_line_indent = len(cont_match.group(1))
                 # 이전 대사 시작 줄보다 들여쓰기가 같거나 크고, 내용이 있다면 연속 대사로 간주
                 if current_line_indent >= last_indent_level:
                     continuation_text = cont_match.group(2).strip()
                     # 혹시 들여쓰기된 줄이 이름 패턴으로 시작하면 제외 (지문 등)
                     if not re.match(r'^[^\s].*?[^\s](?:\s*\([^)]+\))?\s+', continuation_text):
                         cleaned_part = action_re.sub('', continuation_text).strip()
                         if cleaned_part:
                             current_dialogue_lines.append(cleaned_part)
                     else: # 들여쓰기 됐지만 새 이름 패턴이면 이전 대사 저장
                          if current_character and current_dialogue_lines:
                              full_dialogue = " ".join(current_dialogue_lines).strip()
                              full_dialogue = ' '.join(full_dialogue.split())
                              if full_dialogue:
                                  dialogues.append({'Character': current_character, 'Dialogue': full_dialogue})
                          current_character = None
                          current_dialogue_lines = []
                          last_indent_level = -1
                 else: # 들여쓰기가 이전보다 작으면 지문으로 보고 이전 대사 저장
                     if current_character and current_dialogue_lines:
                          full_dialogue = " ".join(current_dialogue_lines).strip()
                          full_dialogue = ' '.join(full_dialogue.split())
                          if full_dialogue:
                              dialogues.append({'Character': current_character, 'Dialogue': full_dialogue})
                     current_character = None
                     current_dialogue_lines = []
                     last_indent_level = -1
            else: # 들여쓰기 없는 줄은 지문으로 보고 이전 대사 저장
                if current_character and current_dialogue_lines:
                     full_dialogue = " ".join(current_dialogue_lines).strip()
                     full_dialogue = ' '.join(full_dialogue.split())
                     if full_dialogue:
                         dialogues.append({'Character': current_character, 'Dialogue': full_dialogue})
                current_character = None
                current_dialogue_lines = []
                last_indent_level = -1
        # else: # 이름 패턴도 아니고, 현재 화자도 없으면 지문 (처리 없음)
        #     pass

    # 마지막 대사 블록 처리
    if current_character and current_dialogue_lines:
        full_dialogue = " ".join(current_dialogue_lines).strip()
        full_dialogue = ' '.join(full_dialogue.split())
        if full_dialogue:
            dialogues.append({'Character': current_character, 'Dialogue': full_dialogue})

    return dialogues

In [108]:
def extract_dialogue_enter_format(script_text):
    """
    주어진 스크립트 텍스트에서 대사만 추출하여 리스트로 반환합니다.
    형식: 이름이 한 줄에 나오고 다음 줄부터 대사가 나오는 형식을 처리합니다.

    Args:
        script_text (str): 영화 대본 텍스트.

    Returns:
        list: [{'Character': 이름, 'Dialogue': 대사}, ...] 형식의 딕셔너리 리스트.
              추출 실패 시 빈 리스트 반환.
    """
    dialogues = []
    lines = script_text.strip().split('\n')

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # 비어있거나 씬 넘버(#숫자. 또는 숫자.) 인 경우 건너뜀
        if not line or line.startswith('#') or re.match(r'^\d+\.\s+', line):
            i += 1
            continue

        # 이름 후보: 주로 한글, 짧은 길이, 콜론(:) 없음, 괄호 설명 가능성 (V.O), (소리) 등
        # 이름으로 간주할 수 있는 패턴 (좀 더 유연하게)
        # - 한글, 영문, 숫자, 공백, 괄호 포함 가능
        # - 너무 길지 않은 라인 (예: 30자 미만)
        # - 명확한 지문 형태가 아닌 경우 (예: '...를 지나치는데-' 로 끝나지 않는 경우)
        potential_name_match = re.match(r'^([^\t\n]{1,30})$', line) # 탭, 개행 제외 1~30자
        is_likely_action = line.endswith('는데-') or line.endswith('다.') or line.endswith('보인다.') or line.endswith('있다.') # 지문일 가능성이 높은 경우 제외

        if potential_name_match and not is_likely_action and (i + 1 < len(lines)):
            potential_character_name = line # 일단 현재 줄을 이름 후보로

            # 다음 줄이 대사인지 확인 (빈 줄이 아니고, 다른 이름/씬넘버 패턴이 아닌 경우)
            next_line_index = i + 1
            while next_line_index < len(lines) and not lines[next_line_index].strip():
                next_line_index += 1 # 빈 줄 건너뛰기

            if next_line_index < len(lines):
                next_line = lines[next_line_index].strip()
                # 다음 줄이 '이름' 후보 패턴과 일치하거나 씬 넘버로 시작하면 현재 줄은 이름이 아님
                is_next_line_name_candidate = bool(re.match(r'^([^\t\n]{1,30})$', next_line)) and not (next_line.endswith('는데-') or next_line.endswith('다.') or next_line.endswith('보인다.') or next_line.endswith('있다.'))
                is_scene_heading = next_line.startswith('#') or re.match(r'^\d+\.\s+', lines[next_line_index]) # 원본 줄 확인

                # 다음 줄이 새로운 이름 후보나 씬넘버가 아니라면 현재 줄을 이름으로 확정하고 대사 추출 시작
                if not is_next_line_name_candidate and not is_scene_heading:
                    character_name = potential_character_name # 이름 확정
                    current_dialogue_lines = []
                    dialogue_line_index = next_line_index

                    while dialogue_line_index < len(lines):
                        dialogue_part_raw = lines[dialogue_line_index] # 원본 줄 유지 (들여쓰기 등 확인 위함)
                        dialogue_part_stripped = dialogue_part_raw.strip()

                        # 다음 줄이 새로운 이름 후보거나 씬 넘버거나 빈 줄이면 대사 끝
                        is_dialogue_name_follow = bool(re.match(r'^([^\t\n]{1,30})$', dialogue_part_stripped)) and not (dialogue_part_stripped.endswith('는데-') or dialogue_part_stripped.endswith('다.') or dialogue_part_stripped.endswith('보인다.') or dialogue_part_stripped.endswith('있다.'))
                        is_scene_heading_follow = dialogue_part_stripped.startswith('#') or re.match(r'^\d+\.\s+', dialogue_part_raw)

                        if not dialogue_part_stripped or (is_dialogue_name_follow and not dialogue_part_raw.startswith('\t') and not dialogue_part_raw.startswith(' ')) or is_scene_heading_follow:
                             # 들여쓰기 없는 이름 후보는 새 이름으로 간주
                            break # 대사 끝

                        # 대사에 포함될 내용 (괄호 안 지문 제거)
                        cleaned_part = re.sub(r'\([^)]*\)', '', dialogue_part_stripped).strip()
                        if cleaned_part:
                            current_dialogue_lines.append(cleaned_part)
                        dialogue_line_index += 1

                    if current_dialogue_lines:
                        full_dialogue = " ".join(current_dialogue_lines).strip()
                        # 여러 공백을 하나로 변경
                        full_dialogue = ' '.join(full_dialogue.split())

                        if full_dialogue:
                            dialogues.append({
                                'Character': character_name,
                                'Dialogue': full_dialogue
                            })
                        i = dialogue_line_index # 다음 처리할 라인 인덱스 업데이트
                        continue # 다음 바깥 루프로

        # '이름 : 대사' 형식도 혹시 모르니 체크 (혼용 가능성)
        dialogue_colon_match = re.match(r'^\s*([^:\n]+?)\s*:\s*(.*)', line)
        if dialogue_colon_match:
             character_name = dialogue_colon_match.group(1).strip()
             dialogue_text = dialogue_colon_match.group(2).strip()
             cleaned_dialogue = re.sub(r'\([^)]*\)', '', dialogue_text).strip()
             cleaned_dialogue = ' '.join(cleaned_dialogue.split())
             if cleaned_dialogue:
                 # 이름만 있는 줄 다음 대사가 오는 형식을 우선하므로, 중복 방지 체크 필요 시 추가
                 # 예: if not dialogues or dialogues[-1]['Character'] != character_name:
                 dialogues.append({
                     'Character': character_name,
                     'Dialogue': cleaned_dialogue
                 })
             i += 1
             continue # 다음 루프로

        # 위 패턴에 모두 맞지 않으면 다음 줄로 이동
        i += 1

    return dialogues

In [109]:
with open('../data/꽃피는봄이오면(2004-02-09).csv', 'r', encoding='utf-8') as f:
    script_content = ''.join(f.readlines())

extracted_dialogues = extract_dialogue_enter_format(script_content)    



# 4. 결과 확인 (예: CSV 형태로 출력)
if extracted_dialogues:
    output = io.StringIO()
    writer = csv.writer(output, quoting=csv.QUOTE_ALL)
    writer.writerow(['Character', 'Dialogue']) # 헤더 작성
    for entry in extracted_dialogues:
        writer.writerow([entry['Character'], entry['Dialogue']])

    csv_output = output.getvalue()
    output.close()
    print("--- CSV 출력 ---")
    print(csv_output)

    # 파일로 저장하고 싶을 경우:
    with open("../data/[대본]꽃피는봄이오면.csv", "w", encoding="utf-8", newline="") as f:
         f.write(csv_output)
    print("\nCSV 파일 '[대본]꽃피는봄이오면.csv 생성 완료.")

else:
    print("추출된 대사가 없습니다. 스크립트 형식을 확인해주세요.")        

--- CSV 출력 ---
"Character","Dialogue"
"S# 1. 구청 관악반 - 오후 (비)","빈 복도. 낡은 창 밖으로 조각 풍경들이 보인다."
"창에 부딪치는 빗방울들 위로 들리는 어색한 연주소리.","""동네 가게 아줌마서부터, 제법 차려입은 예쁜 아줌마들까지 섞여 있는, "" 썰렁한 동네 구청 한 구석의 아줌마들 대상 색소폰 교습반."
"비 내리는 창 밖을 내다보고 있는 현우.","현우 : 영재슈퍼 사모님…. 자꾸 한 박자씩 늦으시네요."
"""현우","연주라는 건, 악기와 몸을 빌려서 마음으로 연주를 하는 거예요. """
"따뜻한 사랑을 상상하세요. 댁에 계시는 남편분들이나.","아줌마들 표정이 갑자기 침울해진다. 그래도 배워보겠다는 자세로 불어보는데 소리가 더 안좋다."
"""현우","아… 아니, 아니, """
"아줌마들 : …….","""현우 : 비 올 때 우산도 씌워주고, 비 맞으면서 기다리고, 같이 걷고…."" 그러던 첫사랑의 추억 다 있으시죠? 남편분들 말구요."
"현우","자 그럼…."
"그 첫사랑을 만진다고 생각하면서 손가락을 움직여보세요.","""“어머 어머, 못하는 소리가 없어 정말” 하면서도, "" 몇몇 아줌마들은 벌써 볼을 붉히며 살며시 입술을 색소폰에 가져간다."
"경수","할만 해?"
"현우","……."
"경수","잘 생각했다. 어머니 고생하시는데."
"현우","표정은 다 장영주야."
"경수","재밌겠네 뭐…."
"현우 : (한숨) … 재미없어.","그냥 아줌마들 좋아하는 얘기 대충 하면서 시간만 때우는 거지. 경수 : 그거라도 감지덕지 해. 요즘 일자리 없어서 난리들인데."
"현우 : …….","현우의 전화가 울린다. 현우는 주머니에서 전화기를 꺼내 발신번호를 잠시 보고 있더니 받지 않는다."
"전화 안받는 현우를 보는 경수.","경수 : 좀 받아라. 넌 뭐 그렇게 죄진 게 많냐?"
"현우","됐어."
"경수","전화 건 사람이 죽어가면서 한 거면 어쩔려구?"
"""현우 : 새끼, 재수 없게….""","전화벨